In [16]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-5-nano")

llm

ChatOpenAI(profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x1162ff750>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x1162ff4d0>, root_client=<openai.OpenAI object at 0x115e916d0>, root_async_client=<openai.AsyncOpenAI object at 0x1162ff250>, model_name='gpt-5-nano', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [17]:
from langchain.tools import tool

@tool
def take_bike():
    """Use bike when traffic is high and rood condition is not good and it's a small trip as well"""
    return f"Bike has been started"

@tool
def take_car():
    """Use car when traffic is low and rood condition is good and it's a long trip as well"""
    return f"Car has been started"

In [25]:
llm = llm.bind_tools([take_bike, take_car], tool_choice="any", parallel_tool_calls=False)

In [26]:
llm.invoke("Its heavy traffic")

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 788, 'prompt_tokens': 163, 'total_tokens': 951, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 768, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cw2nGqdeyZLSMPbZLkWodafWzD3mB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019ba205-0843-77e2-b26e-ab7773803f0a-0', tool_calls=[{'name': 'take_bike', 'args': {}, 'id': 'call_dgeJ51IGgGSbvf1WGssY8jk4', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 163, 'output_tokens': 788, 'total_tokens': 951, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 768}})

In [ ]:
from langgraph.graph import START, END, MessagesState, StateGraph

def call_llm(state:MessagesState):
    return {"messages": llm.invoke(state["messages"])}

def run_tool(state:MessagesState):
    result = []
    tool_calls = state["messages"][-1].tool_calls
    if tool_calls[0]["name"] == "take_bike":
        resp = take_bike.invoke()
    if tool_calls[0]["name"] == "take_car":
        resp = take_car.invoke()
    return result.append({"role": "tool", "tool_call_id": tool_calls[0]["id"], "content" :resp})


def should_continue(state:MessagesState):
  if state["messages"][-1].tool_calls:
    print("tools called...")
    return "run_tool"
  else:
    print("ending...")
    return END



agent = StateGraph(MessagesState)
agent.add_node("call_llm",  call_llm)
agent.add_node("run_tool",  run_tool)

agent.add_edge(START, "call_llm")
agent.add_conditional_edges("call_llm", should_continue, {"run_tool": "runt_tool",  END:END})
agent.add_edge("run_tool", END)

agent = agent.compile()

from IPython.display import Image , display

display()